In [9]:
import random
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os


# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)
SEED = 36

In [10]:
import torchmetrics
from sklearn.utils.class_weight import compute_class_weight

root_dir = 'data'
labels_frame = pd.read_csv('data/labels.csv')

class_weights = compute_class_weight('balanced', classes = labels_frame.label.unique(), y =labels_frame.label)
class_weights = torch.tensor(class_weights, dtype=torch.float32)
num_classes = len(class_weights)
class_weights
config = {
    'num_classes': num_classes,
    'batch_norm': True,
    'batch_size': 64,
    'learning_rate': 0.001,
    'epochs': 1,
    'b1':False,
    'b2':False,
    'root_dir': root_dir,
    'annotation_path':'data/labels.csv',
    'metric':torchmetrics.AUROC(num_classes=num_classes, task='multiclass'),
    'loss_fn': nn.CrossEntropyLoss(weight=class_weights),
    'in_channels': 3
}

In [11]:
class_weights

tensor([0.7630, 0.9761, 1.0144, 1.1574, 1.1759, 1.0857, 1.2310, 0.8122])

In [12]:
import wandb
WANDB_API_KEY = os.environ.get('WANDB_API_KEY')
wandb.login(key=WANDB_API_KEY)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: serhiikravtsov. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\serhii.kravtsov\.netrc


True

In [13]:
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from models.lightning_effnet import EmotionClassifier
from notebooks.lightning_datamodule import EmotionDataModule

def run_train(config):
    run_name = f"effnet_{config['batch_size']}_{config['learning_rate']}_{config['epochs']}"
    wandb.init(project='emotion-recognition', name=run_name, config=config)
    wandb_logger = WandbLogger()
    model = EmotionClassifier(config)
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        dirpath='models',
        filename='emotion_classification_model',
        mode='min',
    )
    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        patience=15,
        verbose=True,
        mode='min'
    )
    trainer = Trainer(max_epochs=config['epochs'], logger=wandb_logger, enable_progress_bar=True,
        enable_checkpointing = True, callbacks=[checkpoint_callback, early_stop_callback])
    annotations_frame = pd.read_csv(config.get('annotation_path'), index_col=0)
    dm = EmotionDataModule(config.get('root_dir'), annotations_frame, config['batch_size'])
    dm.setup(stage='fit')
    trainer.fit(model, dm)
    dm.setup(stage='test')
    model = EmotionClassifier.load_from_checkpoint(checkpoint_callback.best_model_path, config=config)
    trainer.test(model, dm)
    wandb.finish()

In [14]:

run_train(config)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
d:\projects\face-emotion-tracker\venv\Lib\site-packages\pytorch_lightning\loggers\wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
d:\projects\face-emotion-tracker\venv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory D:\projects\face-emotion-tracker\models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\projects\face-emotion-tracker\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.
